In [1]:
from memory_profiler import memory_usage
import time
from collections.abc import Callable
import numpy as np

In [2]:
class GA:
    def __init__(
        self,
        fitness: Callable,
        num_individuals: int,
        num_genes: int,
        num_generations: int,
        tournament_size: int,
        mutation_rate: float,
    ) -> None:
        self.fitness = fitness
        self.rng = np.random.default_rng()
        self.num_individuals = num_individuals
        self.num_genes = num_genes
        self.num_generations = num_generations
        self.tournament_size = tournament_size
        self.mutation_rate = mutation_rate
        self.num_evaluations = 0

    def initialize(self) -> None:
        self.population = [
            [int(self.rng.integers(0, 2)) for _ in range(self.num_genes)]
            for _ in range(self.num_individuals)
        ]

    def evaluation(self, individual: list) -> float:
        return self.fitness(individual)

    def selection(self, tournament_size: int) -> list:
        tournament_winners = []
        for _ in range(self.num_individuals):
            tournament_contestants = self.rng.choice(len(self.population), tournament_size, replace=False)
            tournament_contestants = [self.population[i] for i in tournament_contestants]
            contestants_fitness = []
            for individual in tournament_contestants:
                individual_fitness = self.evaluation(individual)
                self.num_evaluations += 1
                contestants_fitness.append((individual, individual_fitness))
            contestants_fitness.sort(
                key=lambda x: x[1],
                reverse=True,
            )
            tournament_winners.append(contestants_fitness[0][0])
        return tournament_winners

    def crossover(self, selected_parents: list) -> list:
        cut_point = self.rng.integers(1, self.num_genes)

        offspring = []
        for i in range(0, len(selected_parents), 2):
            parent1 = selected_parents[i]
            parent2 = selected_parents[(i + 1) % len(selected_parents)]
            child1 = list(parent1[:cut_point]) + list(parent2[cut_point:])
            child2 = list(parent2[:cut_point]) + list(parent1[cut_point:])
            offspring.append(child1)
            offspring.append(child2)
        return offspring[: self.num_individuals]

    def mutation(self, population: list, mutation_rate: float) -> list:
        mutated_population = []
        for individual in population:
            child = individual.copy()
            mutation_chance = self.rng.random(len(individual))
            for i, rate in enumerate(mutation_chance):
                if rate < mutation_rate:
                    child[i] = 1 - child[i]
            mutated_population.append(child)
        return mutated_population

    def substitute_population(self, population: list) -> None:
        self.population = population

    def optimize(self) -> tuple[list[int], float]:
        self.initialize()
        for _ in range(self.num_generations):
            selected_parents = self.selection(self.tournament_size)
            childs = self.crossover(selected_parents)
            new_population = self.mutation(childs, self.mutation_rate)
            self.substitute_population(new_population)

        best_solution = max(self.population, key=self.evaluation)
        best_fitness = self.evaluation(best_solution)

        return best_solution, best_fitness

In [3]:
def knapsack(binary_vector: list) -> float:
    with open("../data/knapsack_instance.txt") as f:
        linhas = f.read().strip().split("\n")

    max_capacity = int(linhas[1])
    utility_vector = []
    weight_vector = []

    for linha in linhas[2:]:
        a, b = map(int, linha.split())
        utility_vector.append(a)
        weight_vector.append(b)


    fitness = 0
    capacity = 0
    for binary, weight, utility in zip(binary_vector, weight_vector, utility_vector, strict=True):
        fitness += binary * utility
        capacity += binary * weight

    if capacity <= max_capacity:
        return fitness
    return fitness - 10 * (capacity - max_capacity)

In [4]:
with open("../data/knapsack_instance.txt") as f:
    linhas = f.read().strip().split("\n")
    quantity = int(linhas[0])

In [5]:
solutions = []
tempos = []
memorias = []

NUM_INDIVIDUALS = 200
NUM_GENES = quantity
NUM_GENERATIONS = 50
TOURNAMENT_SIZE = 2
MUTATION_RATE = 0.2

for i in range(20):
    def rodar_ga():
        ga = GA(knapsack, NUM_INDIVIDUALS, NUM_GENES, NUM_GENERATIONS, TOURNAMENT_SIZE, MUTATION_RATE)
        best_solution, best_fitness = ga.optimize()
        return ga, best_solution, best_fitness

    start = time.time()
    mem_usage, (ga, best_solution, best_fitness) = memory_usage(
        (rodar_ga,), retval=True, max_usage=True,
    )
    end = time.time()

    solutions.append((best_solution, best_fitness))
    tempos.append(end - start)
    memorias.append(mem_usage)

    print(f"Iteração {i+1}: tempo={end-start:.4f}s, pico de memória={mem_usage:.2f} MiB, avaliações={ga.num_evaluations}, fitness={best_fitness}")

Iteração 1: tempo=1.8563s, pico de memória=106.25 MiB, avaliações=20000, fitness=1748
Iteração 2: tempo=1.6837s, pico de memória=106.25 MiB, avaliações=20000, fitness=1967
Iteração 3: tempo=1.7994s, pico de memória=106.25 MiB, avaliações=20000, fitness=1828
Iteração 4: tempo=1.7861s, pico de memória=106.26 MiB, avaliações=20000, fitness=1861
Iteração 5: tempo=1.7318s, pico de memória=106.26 MiB, avaliações=20000, fitness=2023
Iteração 6: tempo=1.7764s, pico de memória=106.26 MiB, avaliações=20000, fitness=1990
Iteração 7: tempo=1.7864s, pico de memória=106.27 MiB, avaliações=20000, fitness=1899
Iteração 8: tempo=1.7458s, pico de memória=106.27 MiB, avaliações=20000, fitness=1945
Iteração 9: tempo=1.7232s, pico de memória=106.27 MiB, avaliações=20000, fitness=1960
Iteração 10: tempo=1.7325s, pico de memória=106.27 MiB, avaliações=20000, fitness=1939
Iteração 11: tempo=1.6833s, pico de memória=106.27 MiB, avaliações=20000, fitness=1895
Iteração 12: tempo=1.7388s, pico de memória=106.27 M